# Strategy Analysis: Momentum

This notebook provides a detailed analysis of the Momentum trading strategy.

## Strategy Overview

Buy when stock has been going up (positive momentum), sell when stock has been going down (negative momentum).

### Parameters
- Lookback: 90 days
- Allocate: 100% of portfolio


In [ ]:
# Import required libraries
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
from datetime import datetime

# Import our framework
from qb.data import load_csv
from qb.strategy import Momentum
from qb.backtester import Backtester
from qb.metrics import equity_stats

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

## Load Data and Strategy Configuration

In [ ]:
# Load data
ticker = 'GOOGL'
data = load_csv(f'../data/{ticker}.csv')
print(f"Loaded {len(data)} days of {ticker} data")
print(f"Date range: {data.index[0]} to {data.index[-1]}")
print(f"Price range: ${data['Close'].min():.2f} to ${data['Close'].max():.2f}")

# Load strategy configuration
with open('../strategies/momentum.yaml', 'r') as f:
    config = yaml.safe_load(f)

print(f"\nStrategy: {config['name']}")
print(f"Parameters: {config['params']}")
print(f"Initial cash: ${config['initial_cash']:,}")

## Generate Trading Signals

In [ ]:
# Create strategy instance
strategy = Momentum(**config['params'])

# Generate signals
signals = strategy.generate_signals(data)

# Analyze signals
buy_signals = (signals == 1).sum()
sell_signals = (signals == -1).sum()
hold_signals = (signals == 0).sum()

print(f"Signal Analysis:")
print(f"- Buy signals: {buy_signals}")
print(f"- Sell signals: {sell_signals}")
print(f"- Hold signals: {hold_signals}")
print(f"- Total trading days: {buy_signals + sell_signals}")

# Show first few signals
signal_df = pd.DataFrame({
    'Date': data.index,
    'Close': data['Close'],
    'Signal': signals
})
signal_df['Signal_Type'] = signal_df['Signal'].map({1: 'BUY', -1: 'SELL', 0: 'HOLD'})
print(f"\nFirst 10 signals:")
print(signal_df[signal_df['Signal'] != 0].head(10))

## Run Backtest

In [ ]:
# Create backtester
backtester = Backtester(data, strategy, initial_cash=config['initial_cash'])

# Run backtest
results = backtester.run()

# Calculate performance metrics
stats = equity_stats(results['equity'])

print("Performance Metrics:")
print(f"- Total Return: {stats['total_return']:.2%}")
print(f"- Annualized Return: {stats['annualized_return']:.2%}")
print(f"- Volatility: {stats['volatility']:.2%}")
print(f"- Sharpe Ratio: {stats['sharpe']:.3f}")
print(f"- Max Drawdown: {stats['max_drawdown']:.2%}")
print(f"- Win Rate: {stats['win_rate']:.2%}")
print(f"- Profit Factor: {stats['profit_factor']:.2f}")

## Visualize Equity Curve and Signals

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(3, 1, figsize=(15, 12))
fig.suptitle('GOOGL - Momentum Strategy Analysis', fontsize=16, fontweight='bold')

# Plot 1: Price and Signals
axes[0].plot(data.index, data['Close'], label='Close Price', alpha=0.7)
buy_points = data.index[signals == 1]
sell_points = data.index[signals == -1]
axes[0].scatter(buy_points, data.loc[buy_points, 'Close'], 
                color='green', marker='^', s=100, label='Buy Signal', alpha=0.8)
axes[0].scatter(sell_points, data.loc[sell_points, 'Close'], 
                color='red', marker='v', s=100, label='Sell Signal', alpha=0.8)
axes[0].set_title('Price Chart with Trading Signals')
axes[0].set_ylabel('Price ($)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Equity Curve
axes[1].plot(results['equity'].index, results['equity'], label='Portfolio Value', linewidth=2)
axes[1].axhline(y=config['initial_cash'], color='red', linestyle='--', alpha=0.7, label='Initial Investment')
axes[1].set_title('Equity Curve')
axes[1].set_ylabel('Portfolio Value ($)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Plot 3: Drawdown
drawdown = (results['equity'] - results['equity'].cummax()) / results['equity'].cummax()
axes[2].fill_between(drawdown.index, drawdown * 100, 0, alpha=0.3, color='red')
axes[2].plot(drawdown.index, drawdown * 100, color='red', alpha=0.7)
axes[2].set_title('Drawdown')
axes[2].set_ylabel('Drawdown (%)')
axes[2].set_xlabel('Date')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Strategy Summary

### Key Insights:
- [Add insights specific to this strategy]
- [Performance characteristics]
- [Risk/reward profile]

### Recommendations:
- [Parameter optimization suggestions]
- [Market conditions where this strategy works best]
- [Risk management considerations]

### Next Steps:
- Compare with other strategies
- Test on different time periods
- Optimize parameters